<a href="https://colab.research.google.com/github/hiydavid/tfdev_learning/blob/main/DLWP/ch7_working_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working With Keras: A Deep Dive

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

2.8.0


# Three Keras APIs

1. Sequential Model
2. Functional API
3. Model Subclassing

## Sequential Model

In [2]:
# simplest way to build is with keras sequential model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [3]:
# same way but with the .add() method
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [4]:
# calling a model for the first time to build it
model.build(input_shape=(None, 3))

In [5]:
# now the model has weights initiated
model.weights[0][0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([-0.02300069, -0.25901377, -0.11313191, -0.11299528,  0.22816217,
        0.2780074 , -0.0225094 ,  0.24438637,  0.20273018, -0.02197179,
        0.01915857, -0.23261131, -0.22848594, -0.0806995 ,  0.23045135,
        0.25524235, -0.24630018,  0.19013998,  0.21358252, -0.04725441,
        0.08160511,  0.28619808,  0.09339893,  0.10779977, -0.23816442,
        0.28322405,  0.19195598,  0.03112581, -0.1508744 ,  0.10295427,
        0.0417558 , -0.27503386,  0.19479597,  0.05651423,  0.03136498,
       -0.21035156,  0.07397616, -0.00369087, -0.26194674,  0.02933872,
        0.10211536,  0.06388164,  0.22723925, -0.105078  ,  0.21483153,
        0.10508141, -0.02253494,  0.08751619, -0.22420081, -0.03855807,
        0.22516626, -0.2453118 ,  0.21373081, -0.18535164,  0.2857256 ,
        0.07251397, -0.08381939,  0.122529  , -0.13184278,  0.21993679,
        0.05767584,  0.2148791 ,  0.02224407, -0.17222135], dtype=float32)>

In [6]:
# the model also now has a summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                256       
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


In [7]:
# rename model
model = tf.keras.Sequential(name="my_model")
model.add(tf.keras.layers.Dense(64, activation="relu", name="my_frist_layer"))
model.add(tf.keras.layers.Dense(10, activation="softmax", name="my_last_layer"))
model.build(input_shape=(None, 3))
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_frist_layer (Dense)      (None, 64)                256       
                                                                 
 my_last_layer (Dense)       (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


In [8]:
# use Input class to declare input shape which automaticall builds the model
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(3, ))) # sample shape, not batch shape
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [9]:
# can then add new layers and recall summary
model.add(tf.keras.layers.Dense(10, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


## Functional API

In [10]:
# functional version of the previous simple model
inputs = tf.keras.Input(shape=(3, ), name="my_input")
features = tf.keras.layers.Dense(64, activation="relu")(inputs)
outputs = tf.keras.layers.Dense(10, activation="softmax")(features)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [13]:
# input class now holds input information
inputs.shape, inputs.dtype

(TensorShape([None, 3]), tf.float32)

In [14]:
# feature class also has information
features.shape

TensorShape([None, 64])

In [15]:
# the model is instantiated can summary can be viewed
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_6 (Dense)             (None, 64)                256       
                                                                 
 dense_7 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


In [16]:
# a more complex model with multiple inputs and outputs
vocabulary_size = 10000
num_tags = 100
num_departments = 4

# model inputs
title = tf.keras.Input(shape=(vocabulary_size, ), name="title")
text_body = tf.keras.Input(shape=(vocabulary_size, ), name="text_body")
tags = tf.keras.Input(shape=(num_tags, ), name="tags")

# combine input features into a single tensor by concatenating
features = tf.keras.layers.Concatenate()([title, text_body, tags])

# apply an intermediate layer that recombines the features
features = tf.keras.layers.Dense(64, activation="relu")(features)

# define mode outputs
priority = tf.keras.layers.Dense(1, activation="sigmoid", name="priority")(features)
department = tf.keras.layers.Dense(
    num_departments, activation="softmax", name="department")(features)

# create the model
model = tf.keras.Model(
    inputs=([title, text_body, tags]),
    outputs=([priority, department])
)

In [17]:
# check model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title (InputLayer)             [(None, 10000)]      0           []                               
                                                                                                  
 text_body (InputLayer)         [(None, 10000)]      0           []                               
                                                                                                  
 tags (InputLayer)              [(None, 100)]        0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 20100)        0           ['title[0][0]',                  
                                                                  'text_body[0][0]',        

In [21]:
# train this model by provoding lists of input and target arrays
num_samples = 1280

# dummy inputs
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

# dummy outputs
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

# compile this model
model.compile(
    optimizer="rmsprop",
    loss=["mean_squared_error", "categorical_crossentropy"],
    metrics=[["mean_absolute_error"], ["accuracy"]]
)

# fit model
model.fit(
    x=[title_data, text_body_data, tags_data],
    y=[priority_data, department_data],
    epochs=1
)

# evaluate the model
model.evaluate(
    x=[title_data, text_body_data, tags_data],
    y=[priority_data, department_data]
)

# make predictions for each output
priority_preds, department_preds = model.predict(
    x=[title_data, text_body_data, tags_data]
)

40/40 [==============================] - 0s 7ms/step - loss: 33.8031 - priority_loss: 0.3431 - department_loss: 33.4600 - priority_mean_absolute_error: 0.5096 - department_accuracy: 0.1281


In [22]:
# the compile and fit steps can use dicts without relying on order
model.compile(
    optimizer="rmsprop",
    loss={"priority": "mean_squared_error", 
          "department": "categorical_crossentropy"
    },
    metrics={"priority": ["mean_absolute_error"],
             "department": ["accuracy"]
    }
)

model.fit(
    x={"title": title_data,
       "text_body": text_body_data,
       "tags": tags_data
    },
    y={"priority": priority_data, 
       "department": department_data
    },
)

model.evaluate(
    x={"title": title_data,
       "text_body": text_body_data,
       "tags": tags_data
    },
    y={"priority": priority_data, 
       "department": department_data
    }
)

priority_preds, department_preds = model.predict(
    x={"title": title_data, 
       "text_body": text_body_data, 
       "tags": tags_data
    }
)

40/40 [==============================] - 0s 7ms/step - loss: 45.1002 - priority_loss: 0.3431 - department_loss: 44.7571 - priority_mean_absolute_error: 0.5096 - department_accuracy: 0.5492
